In [1]:
from pyvisa import constants,ResourceManager
import time
from scipy.optimize import least_squares
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [25]:
def getFloat(res):
    '''
    Gets a splitted string response to a query and converts what it can to a float
    '''
    for ele in res:
        try:
            number=float(ele)
        except:
            pass
    return number

def getInt(res):
    '''
    Gets a splitted string response to a query and converts what it can to an integer
    '''
    for ele in res:
        try:
            number=int(ele)
        except:
            pass
    return number

def getChunks(hexes,size,vertgain,vertoff):
    '''
    Converts the waveform HEX DAT1 string to values by chunking the string,
    converting each chunk into HEX, then into big endian signed integers, and 
    then using the equation from the manual 
    V = Vertical Gain * INT + Vertical offset
    '''
    chunks = [hexes[i:i+size] for i in range(0,len(hexes),size)]
    #print(chunks)
    vals = []
    for ele in chunks:
        hhex = bytes.fromhex(ele)
        hint = int.from_bytes(hhex,byteorder='big',signed=True)
        #print(hint)
        vals.append(vertgain*hint+vertoff)
    return vals

def getWaveformParams(visa):
    '''
    Gets needed waveform parameters from a series of queries that are converted
    to integer or float depending on the parameters
    '''
    counts = getInt(visa.query('TA:INSPECT? \"WAVE_ARRAY_COUNT\"').split())
    print('Counts:',counts)

    vertgain = getFloat(visa.query('TA:INSPECT? \"VERTICAL_GAIN\"').split())
    print('Vertical gain:',vertgain)
    
    vertoff = getFloat(visa.query('TA:INSPECT? \"VERTICAL_OFFSET\"').split())
    print('Vertical offset:',vertoff)
    
    horint = getFloat(visa.query('TA:INSPECT? \"HORIZ_INTERVAL\"').split())
    print('Horizontal interval:',horint)
    
    horoff = getFloat(visa.query('TA:INSPECT? \"HORIZ_OFFSET\"').split())
    print('Horizontal offset:',horoff)

    return counts,vertgain,vertoff,horint,horoff

def funct(x,t,y):
    return x[0]+x[1]*np.exp(x[2]*t)-y

def gendata(t,a,b,c):
    return a+b*np.exp(t*c)


In [2]:
rm = ResourceManager()
print(rm.list_resources())

('ASRL1::INSTR', 'ASRL2::INSTR', 'ASRL3::INSTR')


In [20]:
lecroy = rm.open_resource('ASRL3::INSTR',
                          baud_rate=112500,
                            #stop_bits = constants.StopBits.two,
                            read_termination = 'fin',
                            write_termination = '\r',
                         )
lecroy.write("CORS EO,\'fin\'")
lecroy.write('CHDR OFF')
lecroy.write('COMM_FORMAT OFF,WORD,HEX')

25

In [4]:
lecroy.write("CORS EO,\'fin\'")

14

In [5]:
lecroy.write('CHL? CLR')

9

In [17]:
lecroy.query('CFMT?')

'\r\nDEF9,WORD,HEX'

In [221]:
lecroy.query('CORS?')

'\r\nEI,13,EO,"'

In [23]:
lecroy.write('TA:FRST')

8

In [26]:
lecroy.write('c1:tra off')

11

In [24]:
lecroy.query('ALST?')

'\r\n\r\nSTB,000000,ESR,000016,INR,008449,DDR,000000,CMR,000000,EXR,000024,URR,000000'

In [9]:
lecroy.query('*IDN?')

'\r\nLECROY,LT372  ,LT37200354,09.3.0'

In [8]:
lecroy.query('TA:INSPECT? \"WAVE_ARRAY_COUNT\"').split()

['TA:INSPECT?', '"WAVE_ARRAY_', '"WAVE_ARRAY_COUNT', ':', '5002', '"']

In [21]:
t1=dt.now()
print(lecroy.query("TA:WF? DAT1").split())
t2=dt.now()
print("it took %.2f seconds" %(t2-t1).total_seconds())

['CORS', 'EO,', 'CH', 'COMM_FORMAT', 'OFF,W', 'TA:WF?', 'D', '...']
it took 1.75 seconds


In [27]:
for i in range(1000):
    try:
        print(lecroy.read())
    except:
        print("nothing further")
        break

nothing further


In [28]:
lecroy.close()

In [ ]:
plt.ion()
fig = plt.figure(figsize=(15,6))
gs = fig.add_gridspec(3,3)
ax1 = fig.add_subplot(gs[:-1,0])
ax1a = fig.add_subplot(gs[-1,0])
ax2 = fig.add_subplot(gs[:-1,1])
ax2a = fig.add_subplot(gs[-1,1])
ax3 = fig.add_subplot(gs[:-1,2])
ax4 = fig.add_subplot(gs[-1,2])
#ax1.set_ylim([0.06,-y_lower])
#ax2.set_ylim([-.05,.05])
ax1a.set_xlabel('$\mu$s')
ax2a.set_xlabel('$\mu$s')
ax4.set_xlabel('Time')
ax1.set_ylabel('Voltage')
                
xs = tdivs[:-1]
ys = [0]*counts[:-1]
                
#ax1.plot(xs,ys,'-k')
#ax1.plot(xs,ys,'-g')
#ax2.plot(xs,ys,'-r')
ax1.grid()
ax1a.grid()
ax2.grid()
ax2a.grid()
ax3.grid()
ax4.grid()

## Resets the trace, waits for ready
ta = dt.datetime.now()
lecroy.write('TA:FRST')
tb = dt.datetime.now()
print((tb-ta).total_seconds())